### Import Packages

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import random
from collections import namedtuple, deque

from unityagents import UnityEnvironment

import matplotlib.pyplot as plt

### Models for the agent:

In [ ]:
class Policy(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Policy, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        
        self.std = nn.Parameter(torch.zeros(action_size))

    def forward(self, state):
        """Build a network that maps state -> action, action distribution (for continuous spaces)."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        mean = F.tanh(self.fc3(x))
        dist = torch.distributions.Normal(mean, F.softplus(self.std))
        action = dist.sample()
        log_prob = dist.log_prob(action).sum(-1).unsqueeze(-1)
        entropy = dist.entropy().sum(-1).unsqueeze(-1)
        #print (log_prob)
        return action, dist, log_prob, entropy
        
        
class Value(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, seed, fc1_units=64, fc2_units=64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Value, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, 1)
        
    def forward(self, state):
        """Build a network that maps state -> value."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)
    
class A2C(nn.Module):
    """A2C Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(A2C, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.actions = nn.Linear(fc2_units, action_size)
        self.value = nn.Linear(fc2_units, 1)
        
        self.std = nn.Parameter(torch.zeros(action_size))

    def forward(self, state):
        """Build a network that maps state -> action, action distribution (for continuous spaces)."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        mean = F.tanh(self.actions(x))
        dist = torch.distributions.Normal(mean, F.softplus(self.std))
        action = dist.sample()
        log_prob = dist.log_prob(action).sum(-1).unsqueeze(-1)
        entropy = dist.entropy().sum(-1).unsqueeze(-1)
        
        value = self.value(x)
        #print (log_prob)
        return action, log_prob, entropy, value

### Actor-Critic Agent

In [ ]:
GAMMA = 0.99            # discount factor
ACTOR_LR = 7e-4         # actor learning rate
CRITIC_LR = 1e-4        # critic learning rate
ROLLOUT_LENGTH = 5      # number of bootstrapping steps

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device('cpu')
print (device)

class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Networks
        #self.actor_network = Policy(state_size, action_size, seed).to(device)
        #self.critic_network = Value(state_size, seed).to(device)
        #self.actor_optimizer = optim.Adam(self.actor_network.parameters(), lr=ACTOR_LR)
        #self.critic_optimizer = optim.Adam(self.critic_network.parameters(), lr=CRITIC_LR)
        
        self.network = A2C(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.network.parameters(), lr=ACTOR_LR)
        
    def step(self, roll_log_probs, roll_entropies, roll_values, roll_rewards, roll_dones):
        """Updates actor network with loss of: advantage * log_probability + entropy
           Updates critic network with loss of: (TD Error)^2
           
        Params
        ======
            states (array_like): current states of all parallel agents
            next_states (array_like): next states of all parallel agents
            actions (array_like): chosen actions of all parallel agents
            dists (Torch object): action distributions of all chosen actions
            rewards (array_like): rewards observed for all parallel agents
            dones (array_like): booleans (True if terminal state, False otherwise)
        """
        gamma = GAMMA
        
        # Initialize tensors
        #print (states)
        #states = torch.from_numpy(states).float().to(device)
        #actions = torch.from_numpy(actions).float().to(device)
        #dists = torch.tensor(states).float().unsqueeze(1).to(device)
        rewards = torch.from_numpy(np.vstack([r for r in rewards])).float().to(device)
        next_states = torch.from_numpy(next_states).float().to(device)
        dones = torch.from_numpy(np.vstack([d for d in dones]).astype(np.uint8)).float().to(device)
                
        # Obtain state values from critic network      
        next_state_values = self.network(next_states)[3]
        #print (next_state_values)
        #state_values = self.critic_network(states)
        state_values = values
        #print (state_values)
        
        # Compute TD Estimates and action advantages
        td_estimates = rewards + (gamma * next_state_values * (1 - dones))
        advantages = td_estimates - state_values
        #print (advantages)
        # Retrieve log probabilities of actions and entropies of action distributions
        #print (dists)
        #print (actions)
        #log_probs = dists.log_prob(actions)
        #print (log_probs)
        #entropies = dists.entropy()
        
        # Compute policy and value losses
        #print (advantages)
        policy_loss = -(log_probs * advantages).mean()
        value_loss = ((advantages)**2).mean()
        total_loss = policy_loss - .01 * entropies.mean() + value_loss
        
        # Optimize actor and critic parameters
        #self.actor_optimizer.zero_grad()
        #policy_loss.backward(retain_graph=True)
        #self.actor_optimizer.step()
        
        #self.critic_optimizer.zero_grad()
        #value_loss.backward()
        #self.critic_optimizer.step()

        self.optimizer.zero_grad()
        total_loss.backward()
        self.optimizer.step()
        
    def act(self, states):
        """Returns action and action distribution for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
        """
        states = torch.from_numpy(states).float().to(device)
        #self.actor_network.eval()
        self.network.eval()
        with torch.no_grad():
            actions, log_probs, entropies, values = self.network(states)
        #self.actor_network.train()
        self.network.train()
        #print (actions)
        actions = actions.squeeze(1)
        actions = actions.cpu().data.numpy()
        #print (actions)                
        #dists = dists.cpu().data.numpy()
        #print (dists)
        return actions, log_probs, entropies, values



Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='Reacher_Windows_x86_64\\Reacher.exe')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Train Agent

In [ ]:
agent = Agent(state_size=state_size, action_size=action_size, seed=0)

def A2C_run(n_episodes=250, max_t=1000/ROLLOUT_LENGTH):
    """Advantage Actor Critic Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
    """
    all_scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)      # last 100 scores
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        for t in range(max_t):
            roll_log_probs = []
            roll_entropies = []
            roll_values = []
            roll_rewards = []
            roll_dones = []
            for _ in ROLLOUT_LENGTH:
                actions, log_probs, entropies, values = agent.act(states)
                roll_log_probs.append(log_probs)
                roll_entropies.append(entropies)
                roll_values.append(values)
                #print (log_probs)
                actions = np.clip(actions, -1, 1)              # all actions between -1 and 1            
                #actions = actions.astype(int)
                env_info = env.step(actions)[brain_name]       # send all actions to the environment
                next_states = env_info.vector_observations         # get next state (for each agent)
                rewards = env_info.rewards                         # get reward (for each agent)
                dones = env_info.local_done                        # see if episode finished
                roll_rewards.append(rewards)
                roll_dones.append(dones)
                scores += env_info.rewards                         # update the score (for each agent)

                states = next_states                               # roll over states to next time step
                if np.any(dones):                                  # exit loop if episode finished
                    break
            agent.step(roll_log_probs, roll_entropies, roll_values, roll_rewards, roll_dones)
        scores_window.append(np.mean(scores))        # save most recent score
        all_scores.append(np.mean(scores))           # save most recent score
        print('\rEpisode {}\tAverage Score: {:.2f}\tLast Score: {:.2f}'.format(i_episode, np.mean(scores_window), np.mean(scores)), end='')
        if i_episode % 10 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}\tLast Score: {:.2f}'.format(i_episode, np.mean(scores_window), np.mean(scores)))
        if np.mean(scores_window)>=30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.actor_network.state_dict(), 'actor_checkpoint.pth')
            torch.save(agent.critic_network.state_dict(), 'critic_checkpoint.pth')
            break
    return all_scores

scores = A2C_run()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 4. Run Agent

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
for _ in range(5):
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    print (actions)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()